## Шаг 1: Защита с помощью Adversarial Training

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist

# Загрузка данных MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Нормализация данных
train_images = train_images / 255.0
test_images = test_images / 255.0

# Преобразование меток в one-hot encoding
train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)

# Уменьшаем количество обучающих изображений для демонстрации
train_images = train_images[:1000]
train_labels = train_labels[:1000]

# Функция FGSM атаки
def fgsm_attack(image, epsilon, gradient):
    perturbation = epsilon * np.sign(gradient)
    adversarial_image = image + perturbation
    adversarial_image = np.clip(adversarial_image, 0, 1)
    return adversarial_image

# Функция для генерации противоречивых примеров Бакин
def generate_adversarial_examples(model, images, labels, epsilon):
    adversarial_images = []
    for i in range(len(images)):
        image = tf.convert_to_tensor(images[i].reshape((1, 28, 28, 1)), dtype=tf.float32)
        label = tf.convert_to_tensor(labels[i].reshape((1, 10)), dtype=tf.float32)
        with tf.GradientTape() as tape:
            tape.watch(image)
            prediction = model(image)
            loss = tf.keras.losses.categorical_crossentropy(label, prediction)
        gradient = tape.gradient(loss, image)
        adversarial_image = fgsm_attack(image.numpy(), epsilon, gradient.numpy())
        adversarial_images.append(adversarial_image.reshape(28, 28))
    return np.array(adversarial_images)

# Создание модели
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Обучение модели с противоречивыми примерами
def adversarial_training(model, train_images, train_labels, epsilon):
    for epoch in range(5):  # Короткое обучение для демонстрации
        for i in range(0, len(train_images), 32):
            batch_images = train_images[i:i+32]
            batch_labels = train_labels[i:i+32]
            adversarial_images = generate_adversarial_examples(model, batch_images, batch_labels, epsilon)
            combined_images = np.concatenate([batch_images, adversarial_images], axis=0)
            combined_labels = np.concatenate([batch_labels, batch_labels], axis=0)
            model.train_on_batch(combined_images, combined_labels)

# Инициализация модели
model = create_model()

# Тренировка модели с защитой на противоречивых примерах
adversarial_training(model, train_images, train_labels, epsilon=0.1)

# Сохранение защищенной модели
model.save('adversarial_trained_model.h5')

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


## Шаг 2: Градиентная маскировка (Gradient Masking)

In [2]:
from tensorflow.keras.layers import Activation

# Обновление модели для градиентной маскировки
def create_masked_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10),
        Activation('softplus')  # Используем softplus вместо softmax для градиентной маскировки
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Обучение модели с градиентной маскировкой
masked_model = create_masked_model()
masked_model.fit(train_images, train_labels, epochs=5)

# Сохранение модели с градиентной маскировкой
masked_model.save('masked_model.h5')

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3695 - loss: 1.9899
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8156 - loss: 0.8792
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8993 - loss: 0.4521
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9426 - loss: 0.2834
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9436 - loss: 0.2591


## Шаг 3: Регуляризация и нормализация

In [3]:
# Модель с регуляризацией и нормализацией
def create_regularized_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Обучение модели с регуляризацией и нормализацией
regularized_model = create_regularized_model()
regularized_model.fit(train_images, train_labels, epochs=5)

# Сохранение модели с регуляризацией
regularized_model.save('regularized_model.h5')

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2577 - loss: 4.3346
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7164 - loss: 2.2442
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7958 - loss: 1.7193
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8401 - loss: 1.5022
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8835 - loss: 1.3061


## Шаг 4: Оценка моделей

In [4]:
# Оценка защищенных моделей на противоречивых примерах
adversarial_test_images = generate_adversarial_examples(model, test_images, test_labels, epsilon=0.1)

print("Base Model Accuracy on Adversarial Examples:") #Базовая модель
model.evaluate(adversarial_test_images, test_labels)

print("Adversarially Trained Model Accuracy:") #Модель с использованием противоречивых примеров
adv_model = tf.keras.models.load_model('adversarial_trained_model.h5')
adv_model.evaluate(adversarial_test_images, test_labels)

print("Masked Model Accuracy:") #Модель с градиентной маскировкой
masked_model = tf.keras.models.load_model('masked_model.h5')
masked_model.evaluate(adversarial_test_images, test_labels)

print("Regularized Model Accuracy:")#Модель с регуляризацией и нормализацией
regularized_model = tf.keras.models.load_model('regularized_model.h5')
regularized_model.evaluate(adversarial_test_images, test_labels)

Base Model Accuracy on Adversarial Examples:
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5013 - loss: 1.4425
Adversarially Trained Model Accuracy:


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5013 - loss: 1.4425
Masked Model Accuracy:


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4470 - loss: 1.6857
Regularized Model Accuracy:


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5517 - loss: 2.2311


[2.12282395362854, 0.6036999821662903]

## Вывод:
Базовая модель (50%):
Базовая модель показала средний уровень точности на противоречивых примерах. Это подтверждает ее уязвимость к атакам и необходимость использования методов защиты.

Модель с использованием противоречивых примеров (50%):
Несмотря на обучающую фазу с учетом противоречивых примеров, результат не улучшился. Это может быть связано с ограниченным количеством данных в тренировочном наборе или недостаточной гибкостью модели.

Модель с градиентной маскировкой (46%):
Градиентная маскировка, реализованная через замену функции активации, не смогла эффективно защитить модель. Это может объясняться тем, что изменение активации ослабляет обучающую способность модели, что приводит к снижению точности.

Модель с регуляризацией и нормализацией (55%):
Наилучший результат достигнут за счет сочетания L2-регуляризации, нормализации и dropout. Эти методы уменьшают переобучение и повышают устойчивость модели, что позволяет ей лучше справляться с противоречивыми примерами.

### Методы защиты моделей дают различный уровень эффективности. Модель с регуляризацией и нормализацией оказалась наиболее простой в реализации и наиболее устойчивой к атакам, что делает ее предпочтительным выбором в данном случае.






